### read keypoint file, and crop visible surfaces 

In [ ]:
import os,re
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
from shapely.geometry import Polygon

In [ ]:
kp_file = "./keypoints-train.txt"  #your keypoint file here
f = open(kp_file,'r')
xlist = []
ylist = []
conflist = []
for line in f:
    line = line.split()
    xtmp = []
    ytmp = []
    conf = []
    for i in range(len(line)):
        if i%3==0:
            xtmp.append(float(line[i]))
        if i%3==1:
            ytmp.append(float(line[i]))
        if i%3==2:
            conf.append(float(line[i])) 
    xlist.append(xtmp)
    ylist.append(ytmp)
    conflist.append(conf)
f.close()

In [ ]:
def line(p1, p2):
    A = (p1[1] - p2[1])
    B = (p2[0] - p1[0])
    C = (p1[0]*p2[1] - p2[0]*p1[1])
    return A, B, -C

def intersection(L1, L2):
    D  = L1[0] * L2[1] - L1[1] * L2[0]
    Dx = L1[2] * L2[1] - L1[1] * L2[2]
    Dy = L1[0] * L2[2] - L1[2] * L2[0]
    if D != 0:
        x = Dx / D
        y = Dy / D
        return x,y
    else:
        return False
def rgb2hsv(r, g, b):
    r, g, b = r/255.0, g/255.0, b/255.0
    mx = max(r, g, b)
    mn = min(r, g, b)
    df = mx-mn
    if mx == mn:
        h = 0
    elif mx == r:
        h = (60 * ((g-b)/df) + 360) % 360
    elif mx == g:
        h = (60 * ((b-r)/df) + 120) % 360
    elif mx == b:
        h = (60 * ((r-g)/df) + 240) % 360
    if mx == 0:
        s = 0
    else:
        s = df/mx
    v = mx
    return h, s, v

In [ ]:
img_path = './image_train/'
track_path = './labeltrain_track/' #save each track in a txt file, one image id in a line
save_path = './frontback/'
K = 3

if not os.path.exists(save_path):
    os.mkdir(save_path)


track_dict = {}
flist = os.listdir(track_path)
if '.DS_Store' in flist:
    del flist[flist.index('.DS_Store')]
for track_file in flist:
    if '.txt' in track_file:
        f = open(track_path+track_file,'r')
        tmp = []
        for line in f:
            tmp.append(re.split('\n',line)[0]+'.jpg')
        track_dict[re.split('.txt',track_file)[0]]=tmp


topK_track_dict = {}
for key,value in track_dict.items():
    trackid = key
    img_size_list = [0,0,0]
    top_k = ["","",""]
    for img in value:
        image = plt.imread（img_path+img)
        size = image.shape[0]*image.shape[1]
        if size > img_size_list[0]:
            top_k[2] = top_k[1]
            top_k[1] = top_k[0]
            top_k[0] = img
            
            img_size_list[2] = img_size_list[1]
            img_size_list[1] = img_size_list[0]
            img_size_list[0] = size
        elif size > img_size_list[1]:
            top_k[2] = top_k[1]
            top_k[1] = img
            img_size_list[2] = img_size_list[1]
            img_size_list[1] = size
        elif size > img_size_list[2]:
            top_k[2] = img
            img_size_list[2] = size
    topK_track_dict[key] = top_k

f=0
scale = 64
front = [15,17,35,33,15]
front_color = [16,17,35,34,16]
back = [10,13,31,28,10]
lside = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,2]
rside = [20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,20]
top = [13,14,32,31,13]
leftfront_wheel = 1
leftback_wheel = 0
rightfront_wheel = 19
rightback_wheel = 18
deglist = []


In [ ]:
#for img in os.listdir(img_path):    
#    if 'jpg' in img:
#f = 0
#for key,value in track_dict.items():
f= 0 
for key,value in topK_track_dict.items():
    trackid = key
    value2 = topK_track_dict[key]
    for img in value:
        image = plt.imread(img_path+img)        
        index = int(re.split('.jpg',img)[0])-1
        x_coor = [x /scale*image.shape[1] for x in xlist[index]]
        y_coor = [y /scale*image.shape[0] for y in ylist[index]]
        conf = conflist[index]        
        
        x_front = []
        y_front = []
        x_front_color = []
        y_front_color = []
        
        x_back = []
        y_back = []
        x_l_side = []
        y_l_side = []
        x_r_side = []
        y_r_side = []
        x_top = []
        y_top = []
        
        for ind in front:
            x_front.append(x_coor[ind])
            y_front.append(y_coor[ind])
        for ind in front_color:
            x_front_color.append(x_coor[ind])
            y_front_color.append(y_coor[ind])
        for ind in back:
            x_back.append(x_coor[ind])
            y_back.append(y_coor[ind])
        for ind in lside:
            x_l_side.append(x_coor[ind])
            y_l_side.append(y_coor[ind])
        for ind in rside:
            x_r_side.append(x_coor[ind])
            y_r_side.append(y_coor[ind])
        for ind in top:
            x_top.append(x_coor[ind])
            y_top.append(y_coor[ind])
            
        #Pfront = Polygon([[x_front[i], y_front[i]] for i in range(len(x_front))])
        #Pback = Polygon([[x_back[i], y_back[i]] for i in range(len(x_back))])
        #Plside = Polygon([[x_l_side[i], y_l_side[i]] for i in range(len(x_l_side))])
        #Prside = Polygon([[x_r_side[i], y_r_side[i]] for i in range(len(x_r_side))])
        #area_front = Pfront.area 
        #area_back = Pback.area 
        #area_lside = Plside.area         
        #area_rside = Prside.area 
        
        ## using opencv mask
        cvPfront = np.array([(x_front[i], y_front[i]) for i in range(len(x_front))], dtype=np.float32)
        cvPfront_color = np.array([(x_front_color[i], y_front_color[i]) for i in range(len(x_front_color))], dtype=np.float32)
        cvPback = np.array([[x_back[i], y_back[i]] for i in range(len(x_back))], dtype=np.float32)
        cvPlside = np.array([[x_l_side[i], y_l_side[i]] for i in range(len(x_l_side))], dtype=np.float32)
        cvPrside = np.array([[x_r_side[i], y_r_side[i]] for i in range(len(x_r_side))], dtype=np.float32)
        
       
        frontrect = cv2.boundingRect(cvPfront)
        frontx,fronty,frontw,fronth = frontrect
        frontcroped = image[fronty:fronty+fronth, frontx:frontx+frontw].copy()
        
        colorfrontrect = cv2.boundingRect(cvPfront_color)
        colorfrontx,colorfronty,colorfrontw,colorfronth = colorfrontrect
        colorfrontcroped = image[colorfronty:colorfronty+colorfronth, colorfrontx:colorfrontx+colorfrontw].copy()
        
        backrect = cv2.boundingRect(cvPback)
        backx,backy,backw,backh = backrect
        backcroped = image[backy:backy+backh, backx:backx+backw].copy()
        
        
        lsiderect = cv2.boundingRect(cvPlside)
        lsidex,lsidey,lsidew,lsideh = lsiderect
        lsidecroped = image[lsidey:lsidey+lsideh, lsidex:lsidex+lsidew].copy()
        
        
        rsiderect = cv2.boundingRect(cvPrside)
        rsidex,rsidey,rsidew,rsideh = rsiderect
        rsidecroped = image[rsidey:rsidey+rsideh, rsidex:rsidex+rsidew].copy() 
        
        
        cvPfront = cvPfront - cvPfront.min(axis=0)
        frontmask = np.zeros(frontcroped.shape[:2], np.uint8)
        #frontdst = cv2.bitwise_and(frontcroped, frontcroped, mask=frontmask)
        
        cvPback = cvPback - cvPback.min(axis=0)
        backmask = np.zeros(backcroped.shape[:2], np.uint8)
        backdst = cv2.bitwise_and(backcroped, backcroped, mask=backmask)
        
        cvPlside = cvPlside - cvPlside.min(axis=0)
        lsidemask = np.zeros(lsidecroped.shape[:2], np.uint8)
        lsidedst = cv2.bitwise_and(lsidecroped, lsidecroped, mask=lsidemask)
        
        cvPrside = cvPlside - cvPrside.min(axis=0)
        rsidemask = np.zeros(rsidecroped.shape[:2], np.uint8)
        rsidedst = cv2.bitwise_and(rsidecroped, rsidecroped, mask=rsidemask)
    
        angle = (np.arctan2((y_coor[leftfront_wheel]+y_coor[rightfront_wheel])/2 - (y_coor[leftback_wheel]+y_coor[rightback_wheel])/2, (x_coor[leftfront_wheel]+x_coor[rightfront_wheel])/2 - (x_coor[leftback_wheel]+x_coor[rightback_wheel])/2)) 
        #angle = np.arctan2((y_coor[leftback_wheel]+y_coor[rightback_wheel])/2 - (y_coor[leftfront_wheel]+y_coor[rightfront_wheel])/2, (x_coor[leftback_wheel]+x_coor[rightback_wheel])/2 - (x_coor[leftfront_wheel]+x_coor[rightfront_wheel])/2)
        #deg = np.rad2deg(angle)
        deg = np.degrees(angle)
        #if angle<0:
        #    angle = -angle
        if deg < 0:
            deg = -deg
        else:
            deg = 360-deg
        deglist.append(deg)
        vis = []    
        if 10<deg<=80:
            ori = 2
            vis = ['rs', 'back']
        elif 80<deg<=100:
            ori = 3
            vis = ['back']
        elif 100<deg<=170:
            ori = 4
            vis = ['ls', 'back']
        elif 170<deg<=190:
            ori = 5 
            vis = ['ls']
        elif 190<deg<=260:
            ori = 6
            vis = ['ls', 'front']
        elif 260<deg<=280:
            ori = 7
            vis = ['front']
        elif 280<deg<=350:
            ori = 8
            vis = ['rs', 'front']
        else:
            ori = 1
            vis = ['rs']
            
        if f < 300:
            #print('leftfront_wheel:',conf[leftfront_wheel])
            #print('leftback_wheel:',conf[leftback_wheel])
            #print('rightfront_wheel:',conf[rightfront_wheel])
            #print('rightback_wheel:',conf[rightback_wheel])
            #print('front_wheel:',(x_coor[leftfront_wheel]+x_coor[rightfront_wheel])/2,(y_coor[leftfront_wheel]+y_coor[rightfront_wheel])/2)
            #print('back_wheel:',(x_coor[leftback_wheel]+x_coor[rightback_wheel])/2,(y_coor[leftback_wheel]+y_coor[rightback_wheel])/2)
            #print("area_front:",area_front)
            #print("area_back:",area_back)
            #print("area_area_lside:",area_lside)
            #print("area_area_rside:",area_rside)
            print(trackid,img,deg,ori,vis)
            plt.figure(figsize=(20,10))
            plt.subplot(131)
            plt.imshow(image)
            #plt.scatter(x_coor,y_coor)
            x_front.append(x_front[0]),y_front.append(y_front[0])
            plt.plot(x_front,y_front,c='red')
            plt.plot(x_back,y_back,c='green')
            plt.plot(x_l_side,y_l_side,c='orange')
            plt.plot(x_r_side,y_r_side,c='purple')
            plt.scatter(x_coor[leftfront_wheel],y_coor[leftfront_wheel])
            plt.scatter(x_coor[leftback_wheel],y_coor[leftback_wheel])
            plt.scatter(x_coor[rightfront_wheel],y_coor[rightfront_wheel])
            plt.scatter(x_coor[rightback_wheel],y_coor[rightback_wheel])
            #plt.scatter((x_coor[leftfront_wheel]+x_coor[rightfront_wheel])/2,(y_coor[leftfront_wheel]+y_coor[rightfront_wheel])/2)
            #plt.scatter((x_coor[leftback_wheel]+x_coor[rightback_wheel])/2,(y_coor[leftback_wheel]+y_coor[rightback_wheel])/2)
            dx = (x_coor[leftfront_wheel]+x_coor[rightfront_wheel])/2 - (x_coor[leftback_wheel]+x_coor[rightback_wheel])/2 
            dy =  (y_coor[leftfront_wheel]+y_coor[rightfront_wheel])/2 - (y_coor[leftback_wheel]+y_coor[rightback_wheel])/2
            plt.arrow((x_coor[leftback_wheel]+x_coor[rightback_wheel])/2,
                      (y_coor[leftback_wheel]+y_coor[rightback_wheel])/2, dx, dy,head_width=5,color='yellow')
            plt.axis('off')
            plt.subplot(132)
            if 'front' in vis:
                plt.imshow(frontcroped)  
                plt.axis('off')
            
            elif 'back' in vis:
                plt.imshow(backcroped)
                plt.axis('off')
            else:
                plt.axis('off')
            plt.subplot(133)
            if 'ls' in vis:
                plt.imshow(lsidecroped)
                plt.axis('off')
            elif 'rs' in vis:
                plt.imshow(rsidecroped)
                plt.axis('off')
                '''
                frontcroped_RGB = cv2.cvtColor(rsidecroped, cv2.COLOR_BGR2RGB)
               # frontcroped_RGB = frontcroped
                data = np.reshape(frontcroped_RGB, (-1,3))
                data = np.float32(data)

                criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
                flags = cv2.KMEANS_RANDOM_CENTERS
                compactness,labels,centers = cv2.kmeans(data,1,None,criteria,10,flags)

                print('Dominant color is: bgr({})'.format(centers[0].astype(np.int32)))
                fig, ax = plt.subplots()
                c =  np.asarray((centers[0].astype(np.int8)))/255
                ax.add_patch(plt.Circle((0.5, 0.5), 0.1, color=c, alpha=1))
               # Use adjustable='box-forced' to make the plot area square-shaped as well.
                ax.set_aspect('equal', adjustable='datalim')
                ax.plot()   #Causes an autoscale update.
                plt.axis("off")
                plt.show()
                '''

            else:
                plt.axis('off')
            plt.show()
            plt.close()
            
        else:
            print("%%%")
            break
    f+=1

In [ ]:
from skimage import transform
from skimage.transform import warp, AffineTransform
from skimage import data
img_path = './image_train/'
save_path = './frontback0429/'

if not os.path.exists(save_path):
    os.mkdir(save_path)

f=0
scale = 64
front = [15,17,35,33]
back = [10,13,31,28]
lside = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
rside = [20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35]
top = [13,14,32,31,13]
leftfront_wheel = 1
leftback_wheel = 0
rightfront_wheel = 19
rightback_wheel = 18
deglist = []


#for img in os.listdir(img_path):
#    if 'jpg' in img:
#        f+=1
for key,value in topK_track_dict.items():
    trackid = key
    all_images_inthistrack = track_dict[key]    
    #for img in all_images_inthistrack:
    n = 0
    
    for img in value:
        n += 1
        f += 1
     #   if img  not in value:
     #       img = value[0]
        if img!= "":
            image = plt.imread(img_path+img)        
            index = int(re.split('.jpg',img)[0])-1
            x_coor = [x /scale*image.shape[1] for x in xlist[index]]
            y_coor = [y /scale*image.shape[0] for y in ylist[index]]
            conf = conflist[index]        

            x_front = []
            y_front = []
            x_back = []
            y_back = []
            x_l_side = []
            y_l_side = []
            x_r_side = []
            y_r_side = []
            x_top = []
            y_top = []

            for ind in front:
                x_front.append(x_coor[ind])
                y_front.append(y_coor[ind])
            for ind in back:
                x_back.append(x_coor[ind])
                y_back.append(y_coor[ind])
            for ind in lside:
                x_l_side.append(x_coor[ind])
                y_l_side.append(y_coor[ind])
            for ind in rside:
                x_r_side.append(x_coor[ind])
                y_r_side.append(y_coor[ind])
            for ind in top:
                x_top.append(x_coor[ind])
                y_top.append(y_coor[ind])

            #Pfront = Polygon([[x_front[i], y_front[i]] for i in range(len(x_front))])
            #Pback = Polygon([[x_back[i], y_back[i]] for i in range(len(x_back))])
            #Plside = Polygon([[x_l_side[i], y_l_side[i]] for i in range(len(x_l_side))])
            #Prside = Polygon([[x_r_side[i], y_r_side[i]] for i in range(len(x_r_side))])
            #area_front = Pfront.area 
            #area_back = Pback.area 
            #area_lside = Plside.area         
            #area_rside = Prside.area 

            ## using opencv mask
            cvPfront = np.array([(x_front[i], y_front[i]) for i in range(len(x_front))], dtype=np.float32)
            cvPback = np.array([[x_back[i], y_back[i]] for i in range(len(x_back))], dtype=np.float32)
            cvPlside = np.array([[x_l_side[i], y_l_side[i]] for i in range(len(x_l_side))], dtype=np.float32)
            cvPrside = np.array([[x_r_side[i], y_r_side[i]] for i in range(len(x_r_side))], dtype=np.float32)


            frontrect = cv2.boundingRect(cvPfront)
            frontx,fronty,frontw,fronth = frontrect
            frontcroped = image[fronty:fronty+fronth, frontx:frontx+frontw].copy()

            backrect = cv2.boundingRect(cvPback)
            backx,backy,backw,backh = backrect
            backcroped = image[backy:backy+backh, backx:backx+backw].copy()


            lsiderect = cv2.boundingRect(cvPlside)
            lsidex,lsidey,lsidew,lsideh = lsiderect
            lsidecroped = image[lsidey:lsidey+lsideh, lsidex:lsidex+lsidew].copy()


            rsiderect = cv2.boundingRect(cvPrside)
            rsidex,rsidey,rsidew,rsideh = rsiderect
            rsidecroped = image[rsidey:rsidey+rsideh, rsidex:rsidex+rsidew].copy() 


            cvPfront = cvPfront - cvPfront.min(axis=0)
            frontmask = np.zeros(frontcroped.shape[:2], np.uint8)
            #frontdst = cv2.bitwise_and(frontcroped, frontcroped, mask=frontmask)

            cvPback = cvPback - cvPback.min(axis=0)
            backmask = np.zeros(backcroped.shape[:2], np.uint8)
            backdst = cv2.bitwise_and(backcroped, backcroped, mask=backmask)

            cvPlside = cvPlside - cvPlside.min(axis=0)
            lsidemask = np.zeros(lsidecroped.shape[:2], np.uint8)
            lsidedst = cv2.bitwise_and(lsidecroped, lsidecroped, mask=lsidemask)

            cvPrside = cvPlside - cvPrside.min(axis=0)
            rsidemask = np.zeros(rsidecroped.shape[:2], np.uint8)
            rsidedst = cv2.bitwise_and(rsidecroped, rsidecroped, mask=rsidemask)



           # frows, fcols = frontcroped.shape[:2]
           # fpts1 =  np.asarray([cvPfront[3],cvPfront[2],cvPfront[0],cvPfront[1]])
           # fpts2 = np.float32([[0, 0], [500,0], [0, 500], [500, 500]])
           # fM = cv2.getPerspectiveTransform(fpts1, fpts2)
            #frontdst = np.flipud(np.rot90(cv2.warpPerspective(frontcroped, fM, (500, 500))))

            #brows, bcols = backcroped.shape[:2]
           # bpts1 =  np.asarray([cvPfront[3],cvPfront[2],cvPfront[0],cvPfront[1]])
           # bpts2 = np.float32([[0, 0], [500,0], [0, 500], [500, 500]])
           # bM = cv2.getPerspectiveTransform(bpts1, bpts2)
           # backdst = np.flipud(np.rot90(cv2.warpPerspective(backcroped, bM, (500, 500))))

            angle = (np.arctan2((y_coor[leftfront_wheel]+y_coor[rightfront_wheel])/2 - (y_coor[leftback_wheel]+y_coor[rightback_wheel])/2, (x_coor[leftfront_wheel]+x_coor[rightfront_wheel])/2 - (x_coor[leftback_wheel]+x_coor[rightback_wheel])/2)) 
            #angle = np.arctan2((y_coor[leftback_wheel]+y_coor[rightback_wheel])/2 - (y_coor[leftfront_wheel]+y_coor[rightfront_wheel])/2, (x_coor[leftback_wheel]+x_coor[rightback_wheel])/2 - (x_coor[leftfront_wheel]+x_coor[rightfront_wheel])/2)
            #deg = np.rad2deg(angle)
            deg = np.degrees(angle)
            #if angle<0:
            #    angle = -angle
            if deg < 0:
                deg = -deg
            else:
                deg = 360-deg
            deglist.append(deg)
            vis = []    
            if 10<deg<=80:
                ori = 2
                vis = ['rs', 'back']
            elif 80<deg<=100:
                ori = 3
                vis = ['back']
            elif 100<deg<=170:
                ori = 4
                vis = ['ls', 'back']
            elif 170<deg<=190:
                ori = 5 
                vis = ['ls']
            elif 190<deg<=260:
                ori = 6
                vis = ['ls', 'front']
            elif 260<deg<=280:
                ori = 7
                vis = ['front']
            elif 280<deg<=350:
                ori = 8
                vis = ['rs', 'front']
            else:
                ori = 1
                vis = ['rs']


            #print(trackid,img,deg,ori,vis)
            plt.imsave(save_path+trackid+'_'+str(n)+'ori.jpg',image)
            if 'front' in vis:
                plt.imsave(save_path+trackid+'_'+str(n)+'front.jpg',frontcroped)
                
            elif 'back' in vis:
                plt.imsave(save_path+trackid+'_'+str(n)+'back.jpg',backcroped)
          
            
            if f%100==0:
                #print("process %d/%d"%(f,len(os.listdir(img_path))))
                print(f)